In [36]:
%load_ext autoreload
%autoreload 2
from slack import SlackInteractor
from llm import LLMInteractor
from runner import process_threads, execute_due_actions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
interactor = SlackInteractor()
llm_interactor = LLMInteractor()

In [38]:
print("\nFetching new messages...")
data = interactor.fetch_new_messages()
threads = interactor.organize_threads(data)
print(f"Found {len(threads)} threads with new messages.")

results = process_threads(interactor, llm_interactor, threads)

print("\nChecking for due actions...")
execute_due_actions(interactor, llm_interactor)



Fetching new messages...
Set conversations_oldest to 1722399936.894109
Loaded 129 old messages
Found 1 new messages
Updated complete_conversations.pkl with new messages
Found 1 threads with new messages.

Processing thread in channel: general
Thread timestamp: 2024-08-08 21:05:30.984519005
Last message content:
Hey everyone! :date: Let's get together for the offsite planning meeting this Monday at 10am. I'll send out a calendar invite shortly so we're all on the same page. :+1:
Debug - Added action due at: 2024-08-08 23:38:47.940661
Debug - Total new actions identified: 1

Raw LLM Response:
{
    "action": {
        "description": "Send out a calendar invite for the offsite planning meeting on Monday at 10am",
        "execution_time": "5 minutes"
    }
}

New actions identified: Send out a calendar invite for the offsite planning meeting on Monday at 10am (Execute at: 2024-08-08 23:38:47.940661)


Checking for due actions...
